In [15]:
# 필요한 패키지 설치 및 임포트
import folium
from folium import Marker
from folium import plugins
from folium import GeoJson
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objects as go

In [16]:
# 서울 스타벅스 지점 데이터 불러오기

url="https://raw.githubusercontent.com/c-karl/DA_DS_Book001/main/datasets/Starbucks_Seoul.csv"
df=pd.read_csv(url)

# 지역 구분을 위한 json 파일 불러오기
geo="https://raw.githubusercontent.com/c-karl/DA_DS_Book001/main/datasets/Seoul_Gu.json"

# 데이터 샘플 확인
df.head()


,name,address,gu_name,latitude,longitude
0,GS타워,서울특별시 강남구 논현로 508 (역삼동),강남구,37.501859,127.037278
1,SSG마켓도곡R,"서울특별시 강남구 언주로30길 57, 타워팰리스Ⅱ F 지하1층 (도곡동)",강남구,37.490298,127.054895
2,W-Mall,서울특별시 금천구 디지털로 188 (가산동),금천구,37.477305,126.887691
3,가든파이브,서울특별시 송파구 충민로 10 (문정동) 가든파이브툴,송파구,37.478232,127.119370
4,가락본동,서울특별시 송파구 송파대로30길 13 (가락동),송파구,37.494895,127.118785


데이터를 판다스 데이터 프레임으로 불러온 다음 확인한다. 각 스타벅스 지점명과 주소, 해당 구, 위도와 경도 정보가 있는 것을 확인할 수 있다. 그리고 지도 위에 시각화를 해야 하기 때문에 서울 각 구의 경계를 표현해줄 수 있는 json 파일도 불러온다.

In [17]:
# 기본 지도 시각화 (서울의 위도, 경도 입력)

m = folium.Map(location=[37.541, 126.986], zoom_start=12)
m

가장 기본적인 지도를 시각화 했다. location= 옵션을 사용하여 나타내고자 하는 지역의 위도와 경도를 입력한다. 그리고 처음 화면에서 어느 정도의 확대 수준으로 시각화 할지는 zoom_start= 옵션을 사용한다.


In [18]:
# 지도 형태 변경
m = folium.Map(location=[37.541, 126.986], tiles='Stamen Toner', zoom_start=12)

# 원하는 좌표에 반경(radius) 표시 (남산)
folium.CircleMarker([37.5538, 126.9810],radius=50, 
                    popup='Laurelhurst Park', color='#3246cc', 
                    fill_color='#3246cc').add_to(m)

# 원하는 좌표에 포인트 표시 (남산)
folium.Marker([37.5538, 126.9810], popup='The Waterfront').add_to(m)
    
m

기본 지도는 옵션을 통하여 형태를 다양하게 바꿀 수 있고 다양한 표시를 삽입할 수 있다. tiles= 옵션은 지도의 형태를 변경해 준다. 예시에서 사용한 Stamen Toner 외에도 Stamen Watercolor, Stamen Terrain, Cartodb Positron, Cartodb Dark_matter 등의 옵션이 있다. 그리고 CircleMarker()와 Marker() 함수를 통해 남산의 위치에 원 표시와 포인트 그림을 삽입했다. 공간 시각화를 할 때 임의로 표시하고 싶은 지점이 있을 때 이러한 기능을 사용한다.

In [19]:
# 서울 지도에 스타벅스 지점 수 시각화

m = folium.Map([37.541, 126.986], zoom_start=12 ,width="%100", height="%100")
locations = list(zip(df.latitude, df.longitude))
cluster = plugins.MarkerCluster(locations=locations,                     
               popups=df["name"].tolist())  
m.add_child(cluster)
m

이제 앞에서 불러왔던 서울 지역의 스타벅스 위치 데이터를 지도 위에 표현한다. MarkerCluster() 함수는 각 구역에 존재하는 스타벅스 지점의 수를 숫자로 표현해 준다. 지도를 더 축소하면 원들이 뭉쳐 표현되면서 숫자들이 합해진다. 반대로 지도를 확대하면 원이 쪼개지다가 결국에는 하나의 핀으로 표현된다.

In [20]:
# 서울 지도에 스타벅스 지점수 도드맵 시각화

m = folium.Map(location=[37.541, 126.986], zoom_start=12, width="%100", height="%100")
locations = list(zip(df.latitude, df.longitude))
for i in range(len(locations)):
    folium.CircleMarker(location=locations[i],radius=1).add_to(m)
m

스타벅스 지점을 도트맵으로 시각화 했다. 서울 중심부와 강남 지역의 큰 길가 그리고 여의도에 매장이 집중되어 포진되어 있는 것을 확인할 수 있다.

In [21]:
# 서울 구 별 스타벅스 지점 수 집계 및 중심점 산출
df_m = df.groupby('gu_name').agg({'latitude':'mean',
                                  'longitude':'mean',
                                  'name':'count'}).reset_index()
df_m.head()

,gu_name,latitude,longitude,name
0,강남구,37.507603,127.044611,80
1,강동구,37.539914,127.137106,14
2,강북구,37.626866,127.026372,5
3,강서구,37.555716,126.841528,16
4,관악구,37.481759,126.944286,11


다음으로 버블맵 시각화를 위해 데이터를 가공한다. 버블맵은 특정 지점에 지정된 수치의 값을 표현하기 때문에 기존 데이터를 구 단위로 집계해야 한다. 우선 gu_name 칼럼을 기준으로 latitude와 longitude의 평균값을 구하여 각 구의 중심 지점을 만든다. 그리고 스타벅스 지점명을 카운트하여 각 구에 포함되는 매장 수 합계를 구한다.

In [22]:
# 서울 구별 스타벅스 지점 수 버블맵 시각화

# 기본 지도 생성
m = folium.Map(location=[37.541, 126.986], tiles='Cartodb Positron', 
               zoom_start=11, width="%100", 
               height="%100")

# 구별 구분선, 색상 설정
folium.Choropleth(
    geo_data=geo, # 앞에서 불러온 json 파일 적용
    fill_color="gray"
    ).add_to(m)

# 버블맵 삽입
locations = list(zip(df_m.latitude, df_m.longitude))
for i in range(len(locations)):
    row = df_m.iloc[i]
    folium.CircleMarker(location=locations[i],
                        radius= float(row.name/2), # 버블 크기 설정
                        fill_color="blue"
                       ).add_to(m)
m

기본 지도에 Cartodb Positron 테마를 적용한 후에 앞에서 불러왔던 json 파일을 통해 구 별 경계선을 표현해 준다. 그리고 CircleMarker() 함수로 버블을 표현해 준다. 버블의 크기를 설정하는 radius=에 매장 수 합계 칼럼인 name을 설정해 주면 매장 수에 따라 버블의 크기가 달라진다. 그리고 버블의 크기가 너무 작아서 안보이거나 너무 커서 서로 겹치면 분모 값을 조정해 주면 된다.

In [23]:
# 미국 실업률 정보의 코로플레스맵 시각화를 위한 데이터, json 불러오기


url2="https://raw.githubusercontent.com/c-karl/DA_DS_Book001/main/datasets/us_states_unemployment.csv"
df2=pd.read_csv(url2)

# 주 별 경계 json 파일 불러오기
us_geo = 'https://raw.githubusercontent.com/c-karl/DA_DS_Book001/main/datasets/folium_us-states.json'

df2.head()

,State,Unemployment
0,AL,7.1
1,AK,6.8
2,AZ,8.1
3,AR,7.2
4,CA,10.1


코로플레스맵은 구역에 따라 채도와 명도가 달라지도록 시각화 하는 방법이기 때문에 단순히 위도와 경도를 설정해서 만들 수 없다. 그래서 앞의 버블맵과 같이 json 파일의 미국 각 주 경계 값경곗값이 필요하다. 그리고 각 주의 위치는 불러온 테이블의 State 칼럼을 매칭해서 Unemployment 값을 표현해 준다.

In [24]:
# 미국 주별 실업률 코로플레스맵 시각화

# 미국 지도 시각화
m = folium.Map(location=[40,-98], zoom_start=3, tiles="Cartodb Positron")

# 지도에 주 경계선, 실업률 데이터 연동
m.choropleth(geo_data = us_geo, # json 데이터
               data = df2, # 실업률 데이터
               columns = ['State','Unemployment'], # 연동할 칼럼 설정
               key_on = 'feature.id', # json과 실업률 데이터를 연결할 키값 설정
               fill_color='YlGn',
               legend_name='실업률')

m

c:\Users\qksw5\anaconda3\envs\test\Lib\site-packages\folium\folium.py:465: FutureWarning:

The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.



기본 지도에 choropleth() 함수를 사용하여 json 데이터와 실업률 데이터를 적용한다. 데이터의 State 칼럼과 json 파일의 id 키 값을 매칭하도록 입력해 준다. 시각화한 결과를 보면 캘리포니아 지역과 노스 캐롤라이나 지역의 실업률이 높은 것을 직관적으로 확인할 수 있다.

In [25]:
# 서울과 각국의 수도 간의 커넥션맵 시각화

# 서울과 도쿄, 워싱턴, 마닐라, 파리, 모스크바 위경도 입력
source_to_dest = zip([37.541,37.541,37.541,37.541,37.541], 
                     [35.6804, 38.9072, 14.5995, 48.8566,55.7558],
                     [126.986,126.986,126.986,126.986,126.986], 
                     [139.7690, -77.0369, 120.9842, 2.3522,37.6173])

fig = go.Figure()

## for 문을 활용하여 위경도 입력
for a, b, c, d in source_to_dest:
    fig.add_trace(go.Scattergeo(
                        lat = [a, b],
                        lon = [c, d],
                        mode = 'lines',
                        line = dict(width = 1, color="red"),
                        opacity = 0.5 # 선 투명도
                        ))

fig.update_layout(
                margin={"t":0,"b":0,"l":0, "r":0, "pad":0},
                showlegend=False,
                geo = dict(
                showcountries=True) # 국가 경계선
                )

fig.show()

커넥션맵은 graph_objects 패키지를 활용하여 테이블 없이 직접 위도와 경도를 입력해서 시각화 해보자. 시작점과 도착점의 위도, 경도를 입력하고 반복문으로 입력했던 모든 경로를 시각화 한다. 서울로부터 도쿄, 워싱턴, 마닐라, 파리, 그리고 모스크바로 연결된 선이 시각화 된 것을 확인할 수 있다.